# Import packages

In [1]:
import os
import src
import src.transforms.aggregate
import pandas as pd
from src import features
from src import database
import yfinance as yf
from config import ROOT_DIR, DB_FILE
from src import extract, transform

# Database

In [2]:
# Create database from csv
database.create_from_csv(
    os.path.join(ROOT_DIR, 'app', 'backend', 'transaction.csv'),
    os.path.join(ROOT_DIR, 'app', 'backend', 'database.db')
)

In [2]:
# List all tables of the database
all_tables = database.list_tables(
    os.path.join(ROOT_DIR, 'app', 'backend', 'database.db')
)
print(all_tables)

['account', 'broker', 'order', 'isin', 'transaction']


# Print dataset

In [3]:
#db_file = "/home/hao/repositories/dashboard-listing-securities/app/backend/database.db"
attribute = "isinId"  # For example, if you want to process the 'accountId' attribute
dict_attribute = extract.by_attribute(DB_FILE, attribute)
df_attribute = pd.DataFrame(dict_attribute)
df_attribute.head()

Successfully connected to /home/hao/repositories/dashboard-listing-securities/app/backend/database.db


,transaction_date,isin,name,type,quantity,unit_price,transaction_price
0,2023-10-16,LU0131510165,Independance et Expansion France Small A,OPVCM,2.0,652.06,1304.12
1,2023-10-16,LU1832174962,Independance et Expansion Europe Small A,OPVCM,15.0,130.30,1954.50
2,2023-10-24,LU1832174962,Independance et Expansion Europe Small A,OPVCM,1.0,126.95,126.95
3,2023-10-24,LU0131510165,Independance et Expansion France Small A,OPVCM,1.0,626.81,626.81
4,2023-11-13,LU1832174962,Independance et Expansion Europe Small A,OPVCM,1.0,131.87,131.87


In [4]:
dict_transactions = extract.all_attribute(DB_FILE)
df_transactions = pd.DataFrame(dict_transactions)
df_transactions.head()

Successfully connected to /home/hao/repositories/dashboard-listing-securities/app/backend/database.db


,transaction_date,isin,isin_name,isin_type,broker_name,broker_country,account_number,account_name,order_type,quantity,unit_price,transaction_price
0,2023-10-16,LU0131510165,Independance et Expansion France Small A,OPVCM,BourseDirect,FRANCE,508TI00083440250EUR,PEA,BUY,2.0,652.06,1304.12
1,2023-10-16,LU1832174962,Independance et Expansion Europe Small A,OPVCM,BourseDirect,FRANCE,508TI00084026141EUR,PEA-PME,BUY,15.0,130.30,1954.50
2,2023-10-24,LU1832174962,Independance et Expansion Europe Small A,OPVCM,BourseDirect,FRANCE,508TI00084026141EUR,PEA-PME,BUY,1.0,126.95,126.95
3,2023-10-24,LU0131510165,Independance et Expansion France Small A,OPVCM,BourseDirect,FRANCE,508TI00083440250EUR,PEA,BUY,1.0,626.81,626.81
4,2023-11-13,LU1832174962,Independance et Expansion Europe Small A,OPVCM,BourseDirect,FRANCE,508TI00084026141EUR,PEA-PME,BUY,1.0,131.87,131.87


In [5]:
z = transform.add_cost_price(dict_transactions)
pd.DataFrame(z)

,isin,isin_name,quantity,cost_price
0,LU0131510165,Independance et Expansion France Small A,9.000000,784.627500
1,LU1832174962,Independance et Expansion Europe Small A,40.000000,695.017500
2,LU1832175001,Independance et Expansion Europe Small I,22.000000,510.815717
3,LU1964632324,Independance et Expansion France Small I,2.000000,914.015000
4,US0846707026,Berkshire Hathaway Inc.,21.666812,254.838520
5,US5705351048,Markel Group Inc.,2.560100,158.068919


In [7]:
# Compute cumulative return for unique isin

# Add last price for ISIN stocks
dict_filter_1 = transform.keep_attribute(
    dict_transactions,
    ['isin']
)
dict_filter_1 = transform.remove_duplicate(dict_filter_1)
dict_filter_1 = transform.add_last_price(dict_filter_1)
df_1 = pd.DataFrame(dict_filter_1)

# Keep only useful attributes
dict_filter_2 = transform.keep_attribute(
    dict_transactions,
    ['transaction_date', 'isin', 'quantity', 'unit_price', 'transaction_price']
)
df_2 = pd.DataFrame(dict_filter_2)

# Merge DataFrames
df = pd.merge(df_2, df_1, on='isin', how='inner')
df

,transaction_date,isin,quantity,unit_price,transaction_price,last_date,last_price
0,2023-10-16,LU0131510165,2.000000,652.06,1304.120000,2024-07-22,769.049988
1,2023-10-16,LU1832174962,15.000000,130.30,1954.500000,2024-07-22,157.119995
2,2023-10-24,LU1832174962,1.000000,126.95,126.950000,2024-07-22,157.119995
3,2023-10-24,LU0131510165,1.000000,626.81,626.810000,2024-07-22,769.049988
4,2023-11-13,LU1832174962,1.000000,131.87,131.870000,2024-07-22,157.119995
...,...,...,...,...,...,...,...
74,2024-07-09,US5705351048,0.138312,1446.00,199.999152,2024-07-25,1577.349976
75,2024-07-16,US0846707026,0.124007,403.20,49.999622,2024-07-25,437.000000
76,2024-07-16,US5705351048,0.135043,1481.00,199.998683,2024-07-25,1577.349976
77,2024-07-23,US0846707026,0.124254,402.40,49.999810,2024-07-25,437.000000


In [16]:
df['current_value'] = df['quantity'] * df['current_price']
df['return'] = (df['current_value'] - df['purchase_value']) / df['purchase_value']
df.head()

,date,isin,quantity,unit_price,purchase_value,current_price,current_value,return
0,2023-10-16,LU0131510165,2.0,652.06,1304.12,773.24,1546.48,0.185842
1,2023-10-16,LU1832174962,15.0,130.30,1954.50,156.19,2342.85,0.198695
2,2023-10-24,LU1832174962,1.0,126.95,126.95,156.19,156.19,0.230327
3,2023-10-24,LU0131510165,1.0,626.81,626.81,773.24,773.24,0.233611
4,2023-11-13,LU1832174962,1.0,131.87,131.87,156.19,156.19,0.184424


In [15]:
total_return_per_action = df.groupby('isin').apply(
    lambda x: (x['current_value'].sum() - x['purchase_value'].sum()) / x['purchase_value'].sum()
)

# Affichage du résultat
total_return_per_action

/tmp/ipykernel_42579/585503647.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  total_return_per_action = df.groupby('isin').apply(


isin
LU0131510165    0.108673
LU1832174962    0.123641
LU1832175001   -0.011704
LU1964632324   -0.048889
US0846707026    0.190332
US5705351048    0.100933
dtype: float64

In [18]:
df[df['isin'] == "LU1964632324"]

,date,isin,quantity,unit_price,purchase_value,current_price,current_value,return
50,2024-04-30,LU1964632324,1.0,890.24,890.24,869.33,869.33,-0.023488
66,2024-05-31,LU1964632324,1.0,937.79,937.79,869.33,869.33,-0.073001


In [7]:
a=src.transforms.aggregate.groupby("isin", dict_transactions)
a

KeyError: "Column(s) ['costPrice'] do not exist"

In [11]:
isin_list = src.transforms.add_attribute.last_price("isin", dict_transactions)
pd.DataFrame(isin_list)

,date,isin,isin_name,isin_type,broker_name,broker_country,account_number,account_name,order_type,quantity,unitPrice,total,previousClosePrice,totalClosePrice
0,2023-10-16,LU0131510165,Independance et Expansion France Small A,OPVCM,BourseDirect,FRANCE,508TI00083440250EUR,PEA,BUY,2.000000,652.06,1304.120000,773.24,1546.480000
1,2023-10-16,LU1832174962,Independance et Expansion Europe Small A,OPVCM,BourseDirect,FRANCE,508TI00084026141EUR,PEA-PME,BUY,15.000000,130.30,1954.500000,156.19,2342.850000
2,2023-10-24,LU1832174962,Independance et Expansion Europe Small A,OPVCM,BourseDirect,FRANCE,508TI00084026141EUR,PEA-PME,BUY,1.000000,126.95,126.950000,156.19,156.190000
3,2023-10-24,LU0131510165,Independance et Expansion France Small A,OPVCM,BourseDirect,FRANCE,508TI00083440250EUR,PEA,BUY,1.000000,626.81,626.810000,773.24,773.240000
4,2023-11-13,LU1832174962,Independance et Expansion Europe Small A,OPVCM,BourseDirect,FRANCE,508TI00084026141EUR,PEA-PME,BUY,1.000000,131.87,131.870000,156.19,156.190000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-07-02,US5705351048,Markel Group Inc.,STOCK,TradeRepublic,GERMANY,0422720001,CTO,BUY,0.136332,1467.00,199.999044,1598.16,217.880349
73,2024-07-09,US0846707026,Berkshire Hathaway Inc.,STOCK,TradeRepublic,GERMANY,0422720001,CTO,BUY,0.132048,378.65,49.999975,434.42,57.364292
74,2024-07-09,US5705351048,Markel Group Inc.,STOCK,TradeRepublic,GERMANY,0422720001,CTO,BUY,0.138312,1446.00,199.999152,1598.16,221.044706
75,2024-07-16,US0846707026,Berkshire Hathaway Inc.,STOCK,TradeRepublic,GERMANY,0422720001,CTO,BUY,0.124007,403.20,49.999622,434.42,53.871121


In [11]:
src.extract.from_yfinance.current_price("LU1964632324")

0P0001IR52.F: Period '1d' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']
/home/hao/repositories/dashboard-listing-securities/src/extract/from_yfinance.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return stock.history(period="1d")['Close'][0]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [5]:
# Example usage
db_file = "/home/hao/repositories/dashboard-listing-securities/app/backend/database.db"
result = from_db.all_attributes(db_file)
pd.DataFrame(result)

NameError: name 'from_db' is not defined

# Load dataset

In [15]:
# Build the full path to the CSV file
csv_file_path = os.path.join(ROOT_DIR, 'app', 'backend', 'transaction.csv')

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

In [16]:
df.head()

,id,isinId,brokerId,accountId,date,orderId,quantity,unitPrice
0,1,1,1,1,2023-10-16,1,2.0,652.06
1,2,3,1,2,2023-10-16,1,15.0,130.30
2,3,3,1,2,2023-10-24,1,1.0,126.95
3,4,1,1,1,2023-10-24,1,1.0,626.81
4,5,3,1,2,2023-11-13,1,1.0,131.87


# Add features

In [6]:
df_features = features.add_column_total(df)
df_features.head()

,id,isinId,brokerId,accountId,date,orderId,quantity,unitPrice,total
0,1,1,1,1,2023-10-16,1,2.0,652.06,1304.12
1,2,3,1,2,2023-10-16,1,15.0,130.30,1954.50
2,3,3,1,2,2023-10-24,1,1.0,126.95,126.95
3,4,1,1,1,2023-10-24,1,1.0,626.81,626.81
4,5,3,1,2,2023-11-13,1,1.0,131.87,131.87


In [9]:
ticker_symbol = 'MKL'  # Replace with the actual ticker symbol corresponding to the ISIN

# Fetching the stock data using yfinance
stock = yf.Ticker('FR0000121014')

# Get the current price
current_price = stock.info['previousClose']
current_price

727.0

In [10]:
import yfinance as yf

# Fetching the stock data using yfinance
stock = yf.Ticker('FR0000121014')

# Get historical market data
hist = stock.history(period='5d')

# Extract the close prices and dates
close_prices = hist['Close'].tolist()
dates = hist.index.tolist()

# Combine the dates and close prices into a list of tuples
recent_closes = list(zip(dates, close_prices))

# Print the results
for date, close in recent_closes:
    print(f"Date: {date}, Close Price: {close}")


Date: 2024-06-21 00:00:00+02:00, Close Price: 717.2999877929688
Date: 2024-06-24 00:00:00+02:00, Close Price: 726.5
Date: 2024-06-25 00:00:00+02:00, Close Price: 736.7999877929688
Date: 2024-06-26 00:00:00+02:00, Close Price: 727.0
Date: 2024-06-27 00:00:00+02:00, Close Price: 715.5


In [11]:
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-06-21 00:00:00+02:00,712.700012,719.500000,710.400024,717.299988,902570,0.0,0.0
2024-06-24 00:00:00+02:00,718.000000,730.200012,716.700012,726.500000,262912,0.0,0.0
2024-06-25 00:00:00+02:00,722.099976,737.799988,721.099976,736.799988,315702,0.0,0.0
2024-06-26 00:00:00+02:00,738.599976,741.799988,720.299988,727.000000,314903,0.0,0.0
2024-06-27 00:00:00+02:00,726.599976,735.599976,714.599976,715.500000,288905,0.0,0.0
